In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from tqdm import tqdm

from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
data = np.load("./data_4/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data_4/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)

train_loader = DataLoader(traindata, batch_size=128, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [24]:
class CRNN(nn.Module):
    def __init__(self, ):
        super(CRNN, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3,  out_channels=64, kernel_size=5, stride=2)
        self.conv2 = nn.Conv1d(
            in_channels=64,  out_channels=64, kernel_size=5, stride=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.5)
        self.rnn = nn.RNN(input_size=3072, hidden_size=100)
        self.fc1 = nn.Linear(100, 2)

    def forward(self, data):
        out = F.relu(self.conv1(data))
        out = F.relu(self.conv2(out))
        out = self.dp(out)
        out = self.pool1(out)
        out = self.fla(out)
        out = out.view(-1, 1, out.shape[-1])
        out, _ = self.rnn(out)
        out = self.fla(out)
        out = self.fc1(out)
        return F.log_softmax(out, dim=1)

In [25]:
%%time
setup_seed(42)
model = CRNN().cuda()
# model=TFEQ(channel=3, time_in=200).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3567
time 1.0 sec:
Test set: Average loss: 0.3527, Accuracy: 1837/2069 (88.79%)

Train Epoch: 2, Average Loss: 0.3512
time 1.0 sec:
Test set: Average loss: 0.3520, Accuracy: 1837/2069 (88.79%)

Train Epoch: 3, Average Loss: 0.3180
time 0.9 sec:
Test set: Average loss: 0.3288, Accuracy: 1902/2069 (91.93%)

Train Epoch: 4, Average Loss: 0.1747
time 0.9 sec:
Test set: Average loss: 0.1500, Accuracy: 1968/2069 (95.12%)

Train Epoch: 5, Average Loss: 0.1284
time 0.9 sec:
Test set: Average loss: 0.1174, Accuracy: 1993/2069 (96.33%)

Train Epoch: 6, Average Loss: 0.0948
time 0.9 sec:
Test set: Average loss: 0.1041, Accuracy: 2007/2069 (97.00%)

Train Epoch: 7, Average Loss: 0.0744
time 0.9 sec:
Test set: Average loss: 0.0881, Accuracy: 2020/2069 (97.63%)

Train Epoch: 8, Average Loss: 0.0623
time 0.9 sec:
Test set: Average loss: 0.0655, Accuracy: 2027/2069 (97.97%)

Train Epoch: 9, Average Loss: 0.0529
time 0.9 sec:
Test set: Average loss: 0.0580, Accuracy: 202

Train Epoch: 73, Average Loss: 0.0095
time 0.9 sec:
Test set: Average loss: 0.0183, Accuracy: 2053/2069 (99.23%)

Train Epoch: 74, Average Loss: 0.0120
time 0.9 sec:
Test set: Average loss: 0.0143, Accuracy: 2057/2069 (99.42%)

Train Epoch: 75, Average Loss: 0.0070
time 0.9 sec:
Test set: Average loss: 0.0175, Accuracy: 2054/2069 (99.28%)

Train Epoch: 76, Average Loss: 0.0074
time 0.9 sec:
Test set: Average loss: 0.0277, Accuracy: 2050/2069 (99.08%)

Train Epoch: 77, Average Loss: 0.0086
time 0.9 sec:
Test set: Average loss: 0.0184, Accuracy: 2055/2069 (99.32%)

Train Epoch: 78, Average Loss: 0.0105
time 0.9 sec:
Test set: Average loss: 0.0122, Accuracy: 2060/2069 (99.57%)

Train Epoch: 79, Average Loss: 0.0060
time 0.9 sec:
Test set: Average loss: 0.0287, Accuracy: 2050/2069 (99.08%)

Train Epoch: 80, Average Loss: 0.0084
time 0.9 sec:
Test set: Average loss: 0.0168, Accuracy: 2057/2069 (99.42%)

Train Epoch: 81, Average Loss: 0.0058
time 0.9 sec:
Test set: Average loss: 0.0148, Accu

Train Epoch: 145, Average Loss: 0.0048
time 0.9 sec:
Test set: Average loss: 0.0111, Accuracy: 2061/2069 (99.61%)

Train Epoch: 146, Average Loss: 0.0044
time 0.9 sec:
Test set: Average loss: 0.0181, Accuracy: 2059/2069 (99.52%)

Train Epoch: 147, Average Loss: 0.0052
time 0.9 sec:
Test set: Average loss: 0.0300, Accuracy: 2054/2069 (99.28%)

Train Epoch: 148, Average Loss: 0.0036
time 0.9 sec:
Test set: Average loss: 0.0163, Accuracy: 2057/2069 (99.42%)

Train Epoch: 149, Average Loss: 0.0058
time 0.9 sec:
Test set: Average loss: 0.0102, Accuracy: 2061/2069 (99.61%)

Train Epoch: 150, Average Loss: 0.0027
time 0.9 sec:
Test set: Average loss: 0.0243, Accuracy: 2056/2069 (99.37%)

Train Epoch: 151, Average Loss: 0.0034
time 0.9 sec:
Test set: Average loss: 0.0156, Accuracy: 2058/2069 (99.47%)

Train Epoch: 152, Average Loss: 0.0042
time 0.9 sec:
Test set: Average loss: 0.0175, Accuracy: 2059/2069 (99.52%)

Train Epoch: 153, Average Loss: 0.0056
time 0.9 sec:
Test set: Average loss: 0.0

In [26]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[1833    8]
 [   4  224]]
Roc score: 0.9891
F1 score: 0.9739
Recall_score: 0.9825
Precision_score: 0.9655
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1841
           1       0.97      0.98      0.97       228

    accuracy                           0.99      2069
   macro avg       0.98      0.99      0.99      2069
weighted avg       0.99      0.99      0.99      2069



In [27]:
torch.save(model, './models/CRNN.pth')

In [6]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [7]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


[  0   0   0   0   0   0   2   1   2 232]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [8]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

[   0    0    0    0    0    1    1    2    2 1827]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
